## Ler arquivos TXT e extrair informações

#### Os arquivos TXT estão incluídos no repositório do Github.
#### Para obter os arquivos txt manualmente , é necessário instalar o programa pdftotext (``sudo apt-get install poppler-utils`` se não estiver instalado) e executar o arquivo ``pdf_to_txt.sh``na pasta ``/pdfs``

Recomendamos rodar este notebook apenas se necessário, pois alguns erros ocorrem devido à falta de uniformidade dos PDFs (ver coluna 'error_flag'). O arquivo ``UC.csv`` foi criado de maneira semiautomática usando este script + ajuste manual, logo quaisquer mudanças nele devem ser feitas com cuidado.

In [1]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import glob
import re


# Path to PDF folder
TXT_PATH = './txts'

e1 = ['nome', lambda X: re.search('Componente Curricular:(.*)\\n',X, re.IGNORECASE).group(1).strip()]
e2  = ['Pré-Requisitos', lambda X: re.search('Pré-requisitos?:(.*)\\n',X,re.IGNORECASE).group(1).strip()]
e3  = ['Carga Horária Total', lambda X: int(re.search('Carga Horária Total:\\D*(\\d+)\\D',X,re.IGNORECASE).group(1))]
e4  = ['Carga Horária Prática', lambda X: int(re.search('Carga Horária.* Prática:\\D*(\\d+)\\D',X,re.IGNORECASE).group(1))]
e5  = ['Carga Horária Teórica', lambda X: int(re.search('Carga Horária.* Teórica:\\D*(\\d+)\\D',X,re.IGNORECASE).group(1))]



def extract_text_info(path):
    """ 
        Reads .txt file from path, then uses regular expressions to fill a dictionary
        with the relevant information
    """
    
    # Get the string used to name the file
    file_ucname =  osp.split(path)[-1][:-4].replace('_', ' ')
    
    info = {'ucname_file':file_ucname , 'error_flag':False}
    
    
    with open(path) as file:
        text = '\r'.join(file.readlines())
    for f in [e1,e2,e3,e4,e5]:
        try:
            info[f[0]] = f[1](text)
        except:
            info['error_flag'] = True
            print(f'{file_ucname}: Failed to get "{f[0]}"')
    return info
    

# We store a list of file dictionaries 
file_dicts = []

for path in glob.glob(osp.join(TXT_PATH,'*.txt')):
    file_dicts.append(extract_text_info(path))
    
    
df = pd.DataFrame.from_records(file_dicts)
for c in ['nome','ucname_file','Pré-Requisitos']:
    df[c] = df[c].str.lower().str.strip().str.replace('.','',regex=False)
for c in df.columns:
    if c.startswith('Carga'):
        df.loc[pd.isna(df[c]), c] = -1
        df[c] = df[c].values.astype(np.int32)

df = df.sort_values('ucname_file')       

Introdução à ecologia: Failed to get "Carga Horária Teórica"
Blendas Poliméricas: Failed to get "Carga Horária Teórica"
 Sustentabilidade no Gerenciamento de Resíduos: Failed to get "Carga Horária Prática"
Processamento de materiais metálicos: Failed to get "Carga Horária Teórica"
Laboratório de matemática: Failed to get "Carga Horária Teórica"
Sistemas distribuídos: Failed to get "nome"
Introdução à bioinformática: Failed to get "Carga Horária Teórica"
Práticas em Projetos Extensionistas II: Failed to get "Carga Horária Teórica"
Prática em Projetos Extensionistas I: Failed to get "Carga Horária Teórica"
Práticas em Instrumentação Biomédica: Failed to get "nome"
Práticas em Instrumentação Biomédica: Failed to get "Pré-Requisitos"
Práticas em Instrumentação Biomédica: Failed to get "Carga Horária Total"
Práticas em Instrumentação Biomédica: Failed to get "Carga Horária Prática"
Práticas em Instrumentação Biomédica: Failed to get "Carga Horária Teórica"
Modelagem de Dados: Failed to get 

___
## Visualizar DataFrame

In [2]:
display(df)

,ucname_file,error_flag,nome,Pré-Requisitos,Carga Horária Total,Carga Horária Prática,Carga Horária Teórica
39,algoritmos e estruturas de dados i,False,algoritmos e estruturas de dados i,lógica de programação,72,36,36
236,algoritmos e estruturas de dados ii,False,algoritmos e estruturas de dados ii,algoritmos e estruturas de dados i,72,26,46
211,algoritmos em bioinformática,False,algoritmos em bioinformática,lógica de programação,72,36,36
190,anatomia,False,anatomia,não há,36,10,26
99,análise de sinais,False,análise de sinais,séries e equações diferenciais ordinárias,72,0,72
...,...,...,...,...,...,...,...
182,validação e verificação de software,False,validação e verificação de software,engenharia de software,72,36,36
113,"vidros, vitrocerâmicas e vidrados",False,"vidros, vitrocerâmicos e vidrados",materiais cerâmicos,72,18,54
41,álgebra linear,False,álgebra linear,geometria analítica,72,10,62
196,álgebra linear computacional,False,álgebra linear computacional,cálculo numérico,72,22,50


---
## Testar se soma de prática e teórica dá a total

In [3]:

b = False == (df['Carga Horária Prática'] + df['Carga Horária Teórica'] == df['Carga Horária Total'])
df.loc[b,'error_flag'] = True
#display(df.loc[b,:])

------
## Testar se nomes são iguais.

In [4]:
b = False == (df['nome']  == df['ucname_file'])
#df.loc[b,'error_flag'] = True
#display(df.loc[b,:])

# Salvar DataFrame em CSV

In [15]:
import csv

df2 = pd.read_csv('UC2.csv',quoting=csv.QUOTE_NONNUMERIC)
assert df2.ucname_file.isin(df.ucname_file).mean() == 1
# Use ucname_file as key
df = df.set_index(keys='ucname_file',drop=False)
df2 = df2.set_index(keys='ucname_file',drop=False)
df['Carga Horária Extensão'] = 0
for c in df2.columns:
    df.loc[df2.index,c] = df2[c]
df.to_csv('UC_manual.csv',index=False,quoting=csv.QUOTE_NONNUMERIC)
display(df)


,ucname_file,error_flag,nome,Pré-Requisitos,Carga Horária Total,Carga Horária Prática,Carga Horária Teórica,Carga Horária Extensão
ucname_file,,,,,,,,
algoritmos e estruturas de dados i,algoritmos e estruturas de dados i,False,algoritmos e estruturas de dados i,lógica de programação,72,36,36,0
algoritmos e estruturas de dados ii,algoritmos e estruturas de dados ii,False,algoritmos e estruturas de dados ii,algoritmos e estruturas de dados i,72,26,46,0
algoritmos em bioinformática,algoritmos em bioinformática,False,algoritmos em bioinformática,lógica de programação,72,36,36,0
anatomia,anatomia,False,anatomia,não há,36,10,26,0
análise de sinais,análise de sinais,False,análise de sinais,séries e equações diferenciais ordinárias,72,0,72,0
...,...,...,...,...,...,...,...,...
validação e verificação de software,validação e verificação de software,False,validação e verificação de software,engenharia de software,72,36,36,0
"vidros, vitrocerâmicas e vidrados","vidros, vitrocerâmicas e vidrados",False,"vidros, vitrocerâmicos e vidrados",materiais cerâmicos,72,18,54,0
álgebra linear,álgebra linear,False,álgebra linear,geometria analítica,72,10,62,0
